Goal: 
   (1) Use Covid Banner information to classify whether the restaurant is temporary closed or not. 
   (2) This is not to predict anything.
   (3) two model are used: SVM and MultinomialNB.
   (4) The text independent variable is preprocess with CountVectory and TFIDF. CountVectory makes the text into a word frequency matrix, TFIDF adjustment the weights of different words in the banner based on their total frequency in all banners.
   

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
os.chdir('/Documents/MSIS/Fall 2020/Team Project')
os.getcwd()

'/Documents/MSIS/Fall 2020/Team Project'

In [4]:
dataset = pd.read_csv('yelpState.csv')
dataset.head()

,business_id,CovidBanner,TemporaryClosedUntil,IsOpen,State
0,--1UhMGODdWsrMastO9DZw,FALSE,FALSE,1,AB
1,--6MefnULPED_I942VcFNA,FALSE,FALSE,1,ON
2,--7zmmkVg-IMGaXbuVd0SQ,FALSE,FALSE,1,NC
3,--8LPVSo5i0Oo61X01sV9A,FALSE,FALSE,1,AZ
4,--9QQLMTbFzLJ_oT-ON3Xw,FALSE,FALSE,1,AZ


Independent variable: Covid Banner is text;  
Dedependent variable: TemporaryClosedUntil is boolean;


In [5]:
train = dataset[dataset['CovidBanner'] != 'FALSE'][['CovidBanner','TemporaryClosedUntil']].dropna()
train.head()

,CovidBanner,TemporaryClosedUntil
26,Pinnacle Restoration is taking Precautions due...,FALSE
28,This location is now open! We're ready to welc...,FALSE
29,We are Open for takeout and delivery .We are f...,FALSE
31,"During this difficult time, we have decided to...",FALSE
63,We are back! This Sport Clips location is offi...,FALSE


In [6]:
train_X = train['CovidBanner']
train_X.head()

26    Pinnacle Restoration is taking Precautions due...
28    This location is now open! We're ready to welc...
29    We are Open for takeout and delivery .We are f...
31    During this difficult time, we have decided to...
63    We are back! This Sport Clips location is offi...
Name: CovidBanner, dtype: object

In [7]:
train.loc[train['TemporaryClosedUntil'] != 'FALSE', 'TemporaryClosedUntil'] = 'TRUE'
train_Y = train['TemporaryClosedUntil']
train_Y.head()

26    FALSE
28    FALSE
29    FALSE
31    FALSE
63    FALSE
Name: TemporaryClosedUntil, dtype: object

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [9]:

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])
text_clf = text_clf.fit(train_X, train_Y)

In [10]:
predicted = text_clf.predict(train_X)
np.mean(predicted == train_Y)

0.9716772490547193

In [22]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge')), ])
text_clf_svm = text_clf_svm.fit(train_X, train_Y)
predicted_svm = text_clf_svm.predict(train_X)
np.mean(predicted_svm == train_Y)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.9789541271313406

In [11]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)), ])
text_clf_svm = text_clf_svm.fit(train_X, train_Y)
predicted_svm = text_clf_svm.predict(train_X)
np.mean(predicted_svm == train_Y)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.9723906684739959

In [12]:
train['predict_svm'] = predicted_svm

In [13]:
train['predict_NNB'] = predicted

In [14]:
train[train['TemporaryClosedUntil'] == 'TRUE'][train['predict_NNB'] == 'TRUE']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,CovidBanner,TemporaryClosedUntil,predict_svm,predict_NNB
67855,Attention - Nos opérations sont actuellement f...,TRUE,FALSE,TRUE
79685,Nos centres sont présentement fermés suite au ...,TRUE,FALSE,TRUE
188713,"Valued Clients,\n\nFollowing the responsible d...",TRUE,FALSE,TRUE


In [15]:
from sklearn import metrics

In [16]:
f1 = metrics.f1_score(train['predict_NNB'], train['TemporaryClosedUntil'], average='macro')
p = metrics.precision_score(train['predict_NNB'], train['TemporaryClosedUntil'], average='macro')
r = metrics.recall_score(train['predict_NNB'], train['TemporaryClosedUntil'], average='macro')
print('Performance for Naive Baysien:')
print("f1:"+str(f1))
print("precision:"+str(p))
print("recall:"+str(r))

Performance for Naive Baysien:
f1:0.5002602086896141
precision:0.503399079777281
recall:0.5800366938075852


In [17]:
f1 = metrics.f1_score(train['predict_svm'], train['TemporaryClosedUntil'], average='macro')
p = metrics.precision_score(train['predict_svm'], train['TemporaryClosedUntil'], average='macro')
r = metrics.recall_score(train['predict_svm'], train['TemporaryClosedUntil'], average='macro')
print('Performance for SVM:')
print("f1:"+str(f1))
print("precision:"+str(p))
print("recall:"+str(r))

Performance for SVM:
f1:0.49300104893840196
precision:0.5
recall:0.48619533423699796


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [21]:
train_Y.describe()

count     14017
unique        2
top       FALSE
freq      13630
Name: TemporaryClosedUntil, dtype: object

Summary:
(1) The training accurary is pretty high such as 0.97
(2) f1, precision and recall are pretty low around 0.5
(3) One important reason for low f1 is because the dataset is very unbalanced. In the dependent training dataset, 97% are false, only 3% are true. 